In [5]:

# -----------------------
# Core / Numerical
# -----------------------
import warnings
import numpy as np
import pandas as pd

from itertools import combinations
from numpy import linalg
from numpy.linalg import norm

# -----------------------
# SciPy / distances
# -----------------------
from scipy.spatial.distance import pdist, squareform

# -----------------------
# Plotting
# -----------------------
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patheffects as PathEffects

# Optional: Seaborn styling with fallback palette
try:
    import seaborn as sns
except ModuleNotFoundError:
    palette = np.array(
        [
            (0.4, 0.7607843137254902, 0.6470588235294118),
            (0.9882352941176471, 0.5529411764705883, 0.3843137254901961),
            (0.5529411764705883, 0.6274509803921569, 0.796078431372549),
            (0.9058823529411765, 0.5411764705882353, 0.7647058823529411),
            (0.6509803921568628, 0.8470588235294118, 0.32941176470588235),
            (1.0, 0.8509803921568627, 0.1843137254901961),
            (0.8980392156862745, 0.7686274509803922, 0.5803921568627451),
            (0.7019607843137254, 0.7019607843137254, 0.7019607843137254),
        ]
    )
else:
    sns.set_style("darkgrid")
    sns.set_palette("muted")
    sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
    palette = np.array(sns.color_palette("Set2"))

# -----------------------
# Scikit-learn
# -----------------------
import sklearn
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import (
    accuracy_score,
    average_precision_score,
    classification_report,
    confusion_matrix,
    f1_score,
    pairwise_distances,
    precision_score,
    recall_score,
    roc_auc_score,
    silhouette_score,
    precision_recall_curve
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, scale
from sklearn.svm import SVC, OneClassSVM
from sklearn.utils import shuffle, resample 

# -----------------------
# PennyLane (Quantum ML)
# -----------------------
import pennylane as qml
import pennylane.numpy as pnp
from pennylane.kernels import kernel_matrix, square_kernel_matrix
from pennylane.optimize import NesterovMomentumOptimizer
import jax
import jax.numpy as jnp


# -----------------------
# Misc
# -----------------------
warnings.filterwarnings("ignore")


In [6]:
RS = 42

In [18]:
# Get Data 
data = pd.read_csv("epb_fiber_dataset.csv")
features = data.drop(["maintenance_flag", "timestamp", 'segment_id'], axis=1)
labels = data["maintenance_flag"]

# Seperate Col Types
cat_cols = ["region_code", "link_type", "vendor", "traffic_period"]
num_cols = ["latency_ms", "latency_jitter_ms", "packet_loss_pct", "error_rate", "reset_count", "throughput_util_pct"]

# One Hot Encode
encoder = OneHotEncoder(sparse_output = False)
encoded = encoder.fit_transform(features[cat_cols])
encoded_data = pd.DataFrame(encoded, columns = encoder.get_feature_names_out(cat_cols), index=data.index)


# Join 
joined_df = pd.concat([encoded_data, features[num_cols]], axis = 1)


# Scale
scaler = StandardScaler()
scaled_df = scaler.fit_transform(joined_df)


final_df = pd.DataFrame(
    scaled_df,
    index=data.index,
    columns = joined_df.columns
    )



In [23]:
# PCA
pca = PCA(n_components=1, random_state=RS)
r1 = pca.fit_transform(final_df)
r1_df = pd.DataFrame(r1, index=data.index)



In [25]:
# Scaling
label_array = labels.to_numpy()
r1_array = r1_df.to_numpy()

x_down1, y_down1 = resample(r1_array, label_array, n_samples=800, replace=False, random_state=RS)
x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(x_down1, y_down1, test_size = .3, stratify=y_down1, random_state=RS)

print(x_train_1.shape)
print(x_train_1[0].shape)

(560, 1)
(1,)


In [26]:
# Initalize Circuit 

# Create Qubits
n_qubits = x_train_1.shape[1] # Select qubits = to the number of columns
dev = qml.device("lightning.qubit", wires=n_qubits) # We are simulating a quantum system with pennylanes default

# Angle Encoding Circut
@qml.qnode(dev)
def circut(x1, x2):
    qml.templates.AngleEmbedding(x1, wires=range(n_qubits)) # template that applies rotation around y axis by default!
    qml.adjoint(qml.templates.AngleEmbedding)(x2, wires=range(n_qubits)) # calculates the hermitian adjoint (inverse)
    return qml.probs(wires=range(n_qubits))



# Create Kernel Funnction with quantum circut
def quantum_kernel(x1, x2):
    return circut(x1, x2)[0]

In [27]:
PCA_kernel_train = square_kernel_matrix(x_train_1, kernel=quantum_kernel)
PCA_kernel_test = kernel_matrix(x_test_1, x_train_1, kernel=quantum_kernel)


KeyboardInterrupt: 

In [ ]:
# Train the SVM
clf1 = SVC(kernel='precomputed', class_weight='balanced')
clf1.fit(PCA_kernel_train, y_train_1)

y_pred = clf1.predict(PCA_kernel_test)

# Metrics
y_scores = clf1.decision_function(PCA_kernel_test)
roc_auc = roc_auc_score(y_test_1, y_scores)
pr_auc = average_precision_score(y_test_1, y_scores)
precision = precision_score(y_test_1, y_pred, average='binary')
recall = recall_score(y_test_1, y_pred, average='binary')
f1 = f1_score(y_test_1, y_pred, average='binary')
accuracy = accuracy_score(y_test_1, y_pred)

print("ROC AUC:", roc_auc)
print("PR AUC:", pr_auc)


print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")